In [14]:
import numpy as np
import pandas as pd
import os

In [15]:
songs = pd.read_csv('../data/songs_short.csv')
user = pd.read_csv('../data/user.csv')
history = pd.read_csv('../data/listening history.csv')


In [16]:
print("Songs head:")
print(songs.head())

print("User head:")
print(user.head())

print("History head:")
print(history.head())

Songs head:
               spotify_id                          Song         Artist  Rank  \
0  4H2TRR9FjnnIwxGnIt9stO                  Killin' Time    Clint Black     1   
1  6iy4PoAuZBMvtrlDX4VxC7               Check Yes Or No  George Strait     2   
2  3oW6SWwGqiZSPTiAp7ZQoH  It's A Great Day To Be Alive   Travis Tritt     3   
3  7lUE02KHkZM44BZgjCaWRO            Meet in the Middle    Diamond Rio     4   
4  1ecIqo8pOc8S6y3l80iGxd                         Fancy  Reba McEntire     5   

     Genre  
0  country  
1  country  
2  country  
3  country  
4  country  
User head:
   ID  Password                 Name Genre 1  Genre 2  Genre 3
0   1         1           Sibyl Bode    rock      r&b      pop
1   2         2    Eleazar Hettinger     rap     soul  hip hop
2   3         3        Geovanny Lang     rap  country     soul
3   4         4    Micah Satterfield    jazz      r&b     soul
4   5         5  Courtney Schowalter    soul     folk    metal
History head:
   User              spoti

In [17]:
# Convert DataFrames to HTML tables for h2o to read
songs_html = songs.to_html(index=False)
user_html = user.to_html(index=False)
history_html = history.to_html(index=False)

# Save HTML tables to files
songs_path = '../data/songs_short.html'
user_path = '../data/user.html'
history_path = '../data/listening history.html'

# Write HTML content to files
with open(songs_path, "w", encoding="utf-8") as f:
    f.write(songs_html)

with open(user_path, "w", encoding="utf-8") as f:
    f.write(user_html)

with open(history_path, "w", encoding="utf-8") as f:
    f.write(history_html)

In [18]:
from h2ogpte import H2OGPTE

In [22]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-ms7SU43E34tS9UJks5RD2KM3m1JumOR2pM73Dk95VzKjM6TZ',
)

# Create Collection
collection_id = client.create_collection(
    name="Musicrecco",
    description="Music Reccomendation Chatbot",
)

# Save HTML tables to files
songs_path = '../data/archive/songs_short.xlsx'
user_path = '../data/archive/user.xlsx'
history_path = '../data/archive/listening history.xlsx'

# Open files for h2o
with open(songs_path, "rb") as f:
    songs_id = client.upload(os.path.basename(songs_path),f)
client.ingest_uploads(collection_id, [songs_id])

with open(user_path, "rb") as f:
    user_id = client.upload(os.path.basename(user_path),f)
client.ingest_uploads(collection_id, [user_id])

with open(history_path, "rb") as f:
    history_id = client.upload(os.path.basename(history_path),f)
client.ingest_uploads(collection_id, [history_id])

Job(id='84d588ba-de66-45a1-a53b-cc981048ff34', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 28, 5, 48, 30, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='36fea923bf79437c86bd632a60267d92', status='Indexing done.'), JobStatus(id='f6f52470dca241bbb96b6ff5d2649e4a', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 28, 5, 48, 35, tzinfo=TzInfo(UTC)), duration='5s', duration_seconds=5.0)

In [23]:
chat_session_id = client.create_chat_session(collection_id)
with client.connect(chat_session_id) as session:
    # Simple Question for Document Collection
        answer = session.query(
        message="what is the top id in users and top id in songs",
        rag_config={
            "rag_type": "rag",
        },
        ).content
        print(answer)


Based on the information provided, the top ID in users is "1" with 4 rock genre listens, and the top ID in songs is "2aoo2jlRnM3A0NyLQqMN2f" with 4 rock genre listens.


Prompt designs

In [ ]:
Prompt_template = """
Answer the questions based on only the following context:

{context}

---

Answer the questions based on the above context: {query}
"""
